In [12]:
import tweepy
import pandas as pd
import itertools
import sys
import csv

### Authorization

In [2]:
consumer_key="FJxeNg2pkApfQkHWp8I0E70gU"
consumer_secret= "es5030A8qeOYvq3jfXmZicq2Yu4X2xOMm0Pfa1z0cqaxeUHqwU"
access_token = "47257922-zzA5oohYNDNN48aUBxNaBPhD6Gme3O8V5BKYlTirk"
access_token_secret = "dmzb11NuClRtuN6r7OLLoWo1AANZjIjgTIX6thBS4cOXh"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

### Exploring Cursor Object

In [9]:
#Search for tweets based on search term
search_term = 'immigration'
number_of_tweets = 1

cursor = tweepy.Cursor(api.search, q= search_term,
                      tweet_mode="extended").items(number_of_tweets)

In [10]:
for i in cursor:
    print(i.full_text)
    print(i.favorite_count)
    print(i.created_at)

RT @DASimmigration: What is the result of procrastination!
Is all of those promisis about immigration limited only to illegal immigrants! S…
0
2021-02-11 20:22:36


### Pulling Tweets with Replies from Single User

In [15]:
def get_all_tweets(screen_name):
    # Twitter only allows access to a users most recent 3240 tweets with this method
    # initialize a list to hold all the tweepy Tweets
    alltweets = []

    # make initial request for most recent tweets (200 is the maximum allowed count)
    new_tweets = api.user_timeline(screen_name=screen_name, count=200)

    # save most recent tweets
    alltweets.extend(new_tweets)

    # save the id of the oldest tweet less one
    oldest = alltweets[-1].id - 1

    # keep grabbing tweets until there are no tweets left to grab
    while len(new_tweets) > 0:
        print
        "getting tweets before %s" % (oldest)

        # all subsiquent requests use the max_id param to prevent duplicates
        new_tweets = api.user_timeline(screen_name=screen_name, count=200, max_id=oldest)

        # save most recent tweets
        alltweets.extend(new_tweets)

        # update the id of the oldest tweet less one
        oldest = alltweets[-1].id - 1

        print
        "...%s tweets downloaded so far" % (len(alltweets))

    # transform the tweepy tweets into a 2D array that will populate the csv
    outtweets = [[tweet.id_str, tweet.created_at, tweet.text.encode("utf-8"), tweet.favorite_count, tweet.retweet_count]
                 for tweet in alltweets]

    # write the csv
    with open('%s_tweets.csv' % screen_name, mode='w', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(["id", "created_at", "text"])
        writer.writerows(outtweets)

    pass

def main():
    get_all_tweets("TuckerCarlson")


if __name__ == '__main__':
    main()

In [24]:


replies=[]
non_bmp_map = dict.fromkeys(range(0x10000, sys.maxunicode + 1), 0xfffd)
for full_tweets in tweepy.Cursor(api.user_timeline,screen_name='TuckerCarlson',timeout=999999).items(1):
  for tweet in tweepy.Cursor(api.search,q='to:'+'TuckerCarlson',result_type='recent',timeout=999999).items(50):
    if hasattr(tweet, 'in_reply_to_status_id_str'):
      if (tweet.in_reply_to_status_id_str==full_tweets.id_str):
        replies.append(tweet.created_at)
        replies.append(tweet.text)

  print("Tweet :",full_tweets.text.translate(non_bmp_map))
  for elements in replies:
       print("Replies :",elements)
  replies.clear()

Tweet : The greatest music of all time. 
https://t.co/1MFa2yxrAX
Replies : 2021-02-11 20:23:49
Replies : @TuckerCarlson @FoxNews #FireTuckerCarlson NOW
Replies : 2021-02-11 20:16:27
Replies : @TuckerCarlson Yeah Tuckuur thinks he is Kurt Cobain in Spain. HAAAA 🤠🤠
Replies : 2021-02-11 19:55:02
Replies : @TuckerCarlson You’re absolutely the lowest form of life. Don’t know how you sleep at night.
Replies : 2021-02-11 19:42:09
Replies : @TuckerCarlson You’re denial of George Floyd’s death is repugnant and grotesque. You’re a noxious entity in our dis… https://t.co/qBWkneQfM1
